In [2]:
import pandas as pd
import pickle as pk
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings("ignore")


In [3]:
value = 4


In [6]:
import os

# Create the directory if it doesn't exist
if not os.path.exists('./csv_files'):
    os.makedirs('./csv_files')


In [7]:
import os
print(os.path.exists('./csv_files/4_poses_data_pose.csv'))


False


In [9]:
import pandas as pd
import numpy as np

# Parameters for generating the dataset
num_samples = 200   # Number of samples you want to generate
num_features = 30   # Assuming 30 key points or features

# Step 1: Generate random feature data
# Using random numbers to simulate key points data, each row representing one yoga pose
features = np.random.rand(num_samples, num_features)

# Step 2: Generate pose labels (e.g., 4 classes of poses labeled as 0, 1, 2, 3)
# Generating synthetic labels for the poses
target = np.random.randint(0, 4, num_samples)

# Step 3: Create a DataFrame with the generated data
data_pose = pd.DataFrame(features, columns=[f'feature_{i}' for i in range(num_features)])
data_pose['pose'] = target  # Adding the target label as 'pose'

# Step 4: Save the generated data to the required CSV path
csv_path = './csv_files/4_poses_data_pose.csv'
data_pose.to_csv(csv_path, index=False)

print(f"CSV file generated and saved at: {csv_path}")


CSV file generated and saved at: ./csv_files/4_poses_data_pose.csv


In [10]:
data_pose = pd.read_csv(f"./csv_files/{value}_poses_data_pose.csv")
features = data_pose.drop(["pose"], axis=1)
target = data_pose[["pose"]]

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2)

data_all_pose_model = RandomForestClassifier()
data_all_pose_model.fit(X_train, y_train)

print(classification_report(y_test, data_all_pose_model.predict(X_test)))


              precision    recall  f1-score   support

           0       0.33      0.67      0.44         6
           1       0.00      0.00      0.00        12
           2       0.00      0.00      0.00         9
           3       0.33      0.23      0.27        13

    accuracy                           0.17        40
   macro avg       0.17      0.22      0.18        40
weighted avg       0.16      0.17      0.16        40



In [12]:

pk.dump(data_all_pose_model, open(f"./models/{value}_poses.model", "wb"))

In [13]:
import pandas as pd
import numpy as np

# Parameters for generating the angles dataset
num_samples = 200  # Number of samples
num_angles = 15    # Assume 15 key angles for each pose (e.g., shoulder, knee, elbow angles)

# Step 1: Generate random angle data (values between 0 and 180 degrees)
angles = np.random.uniform(low=0, high=180, size=(num_samples, num_angles))

# Step 2: Generate target labels for poses (4 possible pose labels: 0, 1, 2, 3)
# These labels represent the type of pose each row corresponds to
pose_labels = np.random.randint(0, 4, num_samples)

# Step 3: Create DataFrame with angles and pose labels
angles_df = pd.DataFrame(angles, columns=[f'angle_{i}' for i in range(num_angles)])
angles_df['pose'] = pose_labels

# Step 4: Save the generated data to the required CSV path
angles_csv_path = './csv_files/4_angles_poses_angles.csv'
angles_df.to_csv(angles_csv_path, index=False)

print(f"CSV file generated and saved at: {angles_csv_path}")


CSV file generated and saved at: ./csv_files/4_angles_poses_angles.csv


In [1]:
import mediapipe as mp
import cv2
import pandas as pd
import numpy as np
from calc_angles import rangles
import os


In [2]:

mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

# Define column names for the pose landmarks CSV (each landmark has x, y, z, visibility)
landmark_columns = [
    f'{name}_{dim}' for name in [
        'nose', 'left_eye_inner', 'left_eye', 'left_eye_outer',
        'right_eye_inner', 'right_eye', 'right_eye_outer', 'left_ear', 'right_ear',
        'left_shoulder', 'right_shoulder', 'left_elbow', 'right_elbow',
        'left_wrist', 'right_wrist', 'left_pinky', 'right_pinky', 'left_index', 'right_index',
        'left_thumb', 'right_thumb', 'left_hip', 'right_hip', 'left_knee', 'right_knee',
        'left_ankle', 'right_ankle', 'left_heel', 'right_heel', 'left_foot_index', 'right_foot_index'
    ] for dim in ['x', 'y', 'z', 'v']
]


In [3]:
# List to store landmarks data for each frame
landmarks_list = []

# Set up video capture (use an existing video file)
video_path = "/home/ubuntu/v2/yoga/sy-1.mp4"
cap = cv2.VideoCapture(video_path)

# Set up MediaPipe Pose
with mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Convert the image color from BGR to RGB as MediaPipe uses RGB
        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(image_rgb)

        # Check if pose landmarks are detected
        if results.pose_landmarks:
            # Extract the landmark data
            landmarks = results.pose_landmarks.landmark
            frame_landmarks = []

            # Extract x, y, z, visibility for each landmark
            for landmark in landmarks:
                frame_landmarks.extend([landmark.x, landmark.y, landmark.z, landmark.visibility])

            landmarks_list.append(frame_landmarks)

# Release the video capture
cap.release()

# Create DataFrame from landmarks list
pose_landmarks_df = pd.DataFrame(landmarks_list, columns=landmark_columns)

# Save to CSV
pose_landmarks_csv_path = "./csv_files/4_poses_data_pose.csv"
os.makedirs('./csv_files', exist_ok=True)
pose_landmarks_df.to_csv(pose_landmarks_csv_path, index=False)

print(f"Pose landmarks CSV file generated and saved at: {pose_landmarks_csv_path}")


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1732205087.251968    9033 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1732205087.311528    9033 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1732205087.355151    9034 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.


ValueError: 124 columns passed, passed data had 132 columns

In [4]:
# Define column names for the pose landmarks DataFrame correctly
landmark_columns = [
    f'{name}_{dim}' for name in [
        'nose', 'left_eye_inner', 'left_eye', 'left_eye_outer',
        'right_eye_inner', 'right_eye', 'right_eye_outer', 
        'left_ear', 'right_ear', 'left_shoulder', 'right_shoulder', 
        'left_elbow', 'right_elbow', 'left_wrist', 'right_wrist', 
        'left_pinky', 'right_pinky', 'left_index', 'right_index',
        'left_thumb', 'right_thumb', 'left_hip', 'right_hip', 
        'left_knee', 'right_knee', 'left_ankle', 'right_ankle', 
        'left_heel', 'right_heel', 'left_foot_index', 'right_foot_index'
    ] for dim in ['x', 'y', 'z', 'v']
]

# The above list should produce exactly 132 columns (33 landmarks * 4 dimensions)
print(f"Number of columns defined in landmark_columns: {len(landmark_columns)}")


Number of columns defined in landmark_columns: 124


In [5]:
# Define the correct column names for the pose landmarks DataFrame
landmark_columns = [
    f'{name}_{dim}' for name in [
        'nose', 'left_eye_inner', 'left_eye', 'left_eye_outer',
        'right_eye_inner', 'right_eye', 'right_eye_outer', 
        'left_ear', 'right_ear', 'mouth_left', 'mouth_right',
        'left_shoulder', 'right_shoulder', 
        'left_elbow', 'right_elbow',
        'left_wrist', 'right_wrist', 
        'left_pinky', 'right_pinky', 
        'left_index', 'right_index',
        'left_thumb', 'right_thumb', 
        'left_hip', 'right_hip', 
        'left_knee', 'right_knee', 
        'left_ankle', 'right_ankle', 
        'left_heel', 'right_heel', 
        'left_foot_index', 'right_foot_index'
    ] for dim in ['x', 'y', 'z', 'v']
]

# The above list should produce exactly 132 columns (33 landmarks * 4 dimensions)
print(f"Number of columns defined in landmark_columns: {len(landmark_columns)}")


Number of columns defined in landmark_columns: 132


In [6]:
# Assuming you've already extracted landmarks and appended to landmarks_list
# Create DataFrame from landmarks list with the correct number of columns
pose_landmarks_df = pd.DataFrame(landmarks_list, columns=landmark_columns)

# Save to CSV
pose_landmarks_csv_path = "./csv_files/4_poses_data_pose.csv"
os.makedirs('./csv_files', exist_ok=True)
pose_landmarks_df.to_csv(pose_landmarks_csv_path, index=False)

print(f"Pose landmarks CSV file generated and saved at: {pose_landmarks_csv_path}")


Pose landmarks CSV file generated and saved at: ./csv_files/4_poses_data_pose.csv


In [7]:
import pandas as pd
from calc_angles import rangles


In [8]:
# Load the previously generated landmarks CSV
pose_landmarks_csv_path = "./csv_files/4_poses_data_pose.csv"
pose_landmarks_df = pd.read_csv(pose_landmarks_csv_path)


In [9]:
# List to store calculated angles for each sample/frame
angles_list = []

# Iterate over each row in the pose landmarks DataFrame to calculate angles
for _, row in pose_landmarks_df.iterrows():
    # Creating a dictionary for landmark points (x, y, z coordinates)
    landmarks_points = {
        landmark_name: [row[f"{landmark_name}_x"], row[f"{landmark_name}_y"], row[f"{landmark_name}_z"]]
        for landmark_name in set(col.split('_')[0] for col in pose_landmarks_df.columns if '_x' in col)
    }

    # Calculate the angles using the rangles function from calc_angles.py
    angles = rangles(pose_landmarks_df, landmarks_points)
    angles_list.append(angles)

# Step 4: Create a DataFrame for the calculated angles
# Define the columns for angles based on what `rangles()` returns
angle_columns = [
    "armpit_left", "armpit_right", "elbow_left", "elbow_right",
    "hip_left", "hip_right", "knee_left", "knee_right",
    "ankle_left", "ankle_right"
]

angles_df = pd.DataFrame(angles_list, columns=angle_columns)

# Step 5: Save the calculated angles to a CSV file
angles_csv_path = "./csv_files/4_angles_poses_angles.csv"
angles_df.to_csv(angles_csv_path, index=False)

print(f"Angles CSV file generated and saved at: {angles_csv_path}")


KeyError: 'left_x'

In [11]:
# List to store calculated angles for each sample/frame
angles_list = []

# List of all landmarks as defined in the original extraction
landmark_names = [
    'nose', 'left_eye_inner', 'left_eye', 'left_eye_outer',
    'right_eye_inner', 'right_eye', 'right_eye_outer', 
    'left_ear', 'right_ear', 'mouth_left', 'mouth_right',
    'left_shoulder', 'right_shoulder', 
    'left_elbow', 'right_elbow',
    'left_wrist', 'right_wrist', 
    'left_pinky', 'right_pinky', 
    'left_index', 'right_index',
    'left_thumb', 'right_thumb', 
    'left_hip', 'right_hip', 
    'left_knee', 'right_knee', 
    'left_ankle', 'right_ankle', 
    'left_heel', 'right_heel', 
    'left_foot_index', 'right_foot_index'
]

# Iterate over each row in the pose landmarks DataFrame to calculate angles
for _, row in pose_landmarks_df.iterrows():
    landmarks_points = {}

    # Extract x, y, z for each landmark using the correct landmark names
    for landmark_name in landmark_names:
        try:
            landmarks_points[landmark_name] = [
                row[f"{landmark_name}_x"], 
                row[f"{landmark_name}_y"], 
                row[f"{landmark_name}_z"]
            ]
        except KeyError:
            # Print warning if any landmark is missing
            print(f"Warning: Missing data for landmark {landmark_name}")
            continue

    # Calculate the angles using the rangles function from calc_angles.py
    angles = rangles(pose_landmarks_df, landmarks_points)
    angles_list.append(angles)

# Step 4: Create a DataFrame for the calculated angles
# Define the columns for angles based on what `rangles()` returns
angle_columns = [
    "armpit_left", "armpit_right", "elbow_left", "elbow_right",
    "hip_left", "hip_right", "knee_left", "knee_right",
    "ankle_left", "ankle_right"
]

# Create DataFrame for the calculated angles
angles_df = pd.DataFrame(angles_list, columns=angle_columns)

# Step 5: Save the calculated angles to a CSV file
angles_csv_path = "./csv_files/4_angles_poses_angles.csv"
angles_df.to_csv(angles_csv_path, index=False)

print(f"Angles CSV file generated and saved at: {angles_csv_path}")


Angles CSV file generated and saved at: ./csv_files/4_angles_poses_angles.csv
